In [1]:
import requests,configparser
from IPython.display import display


In [2]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [3]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [12]:
r = requests.get(getBaseURL()+"&q=SELECT DISTINCT taxonomicgroup_submitted FROM sgcn.sgcn WHERE taxonomicgroup_submitted <> '' ORDER BY taxonomicgroup_submitted").json()

for feature in r["features"]:
    tg = feature["properties"]["taxonomicgroup_submitted"]
    if tg == "Amphibian":
        tgMapping = '"Amphibian"=>"Aphibians"'
    elif tg == "Bird"
    
    

Algae
Amphibian
Amphibians
Arachnids
Arthropod
Bicolor Sand Shrimp
Bird
Birds
Bivalves
Bryophytes
Cephalopods
Cnidarians
Crustaceans
Echinoderms
Fish
Flatworm
Freshwater Mussel
Fungi
Gastropods
Insect
Insects
Invertebrate
Invertebrates
Mammal
Mammals
Mite
Mollusks
Mussel
Myriapods
Other Invertebrates
Plant
Plants
Poriferans
Reptile
Reptiles
Snail
Spider
Vascular Plants
Worms


In [13]:
tgMappings = ['"Amphibian"=>"Amphibians"','"Bird"=>"Birds"','"Insect"=>"Insects"','"Invertebrate"=>"Other Invertebrates"','"Invertebrates"=>"Other Invertebrates"','"Mammal"=>"Mammals"','"Plant"=>"Plants"','"Reptile"=>"Reptiles"','"Vascular Plants"=>"Plants"','"Spider"=>"Arachnids"']
for tgMapping in tgMappings:
    print (tgMapping)


"Amphibian"=>"Amphibians"
"Bird"=>"Birds"
"Insect"=>"Insects"
"Invertebrate"=>"Other Invertebrates"
"Invertebrates"=>"Other Invertebrates"
"Mammal"=>"Mammals"
"Plant"=>"Plants"
"Reptile"=>"Reptiles"
"Vascular Plants"=>"Plants"
"Spider"=>"Arachnids"
